# Module 0: Simple RAG with LangSmith Tracing

This personalized notebook sets up a small RAG app using LangSmith docs and adds tracing configured to your own project. Replace placeholders below with your keys if not using a `.env`.



In [2]:
# Setup
import os
from dotenv import load_dotenv

# Inline env (optional). Prefer .env in project root.
os.environ.setdefault("LANGSMITH_TRACING", "true")
os.environ.setdefault("LANGSMITH_PROJECT", "Langsmith_intro")  # personalize project name
os.environ.setdefault("GEMINI_API_KEY", os.getenv("GEMINI_API_KEY", ""))
os.environ.setdefault("LANGSMITH_API_KEY", os.getenv("LANGSMITH_API_KEY", ""))

# Load from .env if present
load_dotenv(dotenv_path="../../.env", override=True)
print("Tracing:", os.getenv("LANGSMITH_TRACING"), "Project:", os.getenv("LANGSMITH_PROJECT"))

# Install Gemini client if needed
try:
    import google.generativeai as genai  # noqa: F401
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "google-generativeai", "-q"])


Tracing: true Project: nischala-personal-project


In [4]:

# Personalized RAG application
from langsmith import traceable
from typing import List
import nest_asyncio

# Reuse utils retriever from module_0
import sys
sys.path.append("../notebooks/module_0")
from utils import get_vector_db_retriever

# Gemini setup
import os
import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY", ""))
MODEL_PROVIDER = "google"
MODEL_NAME = "models/gemini-2.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = (
    "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the latest question in the conversation. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."
)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="retriever", metadata={"owner": "Nischala", "vectordb": "sklearn"})
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain", metadata={"app_version": APP_VERSION})
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {"role": "system", "content": RAG_SYSTEM_PROMPT},
        {"role": "user", "content": f"Context: {formatted_docs} \n\n Question: {question}"},
    ]
    return call_gemini(messages)

@traceable(run_type="llm", metadata={"ls_provider": MODEL_PROVIDER, "ls_model_name": MODEL_NAME})
def call_gemini(messages: List[dict]):
    # Convert OpenAI-like messages to a single prompt for Gemini
    prompt = "\n\n".join([m["content"] for m in messages])
    model = genai.GenerativeModel(MODEL_NAME)
    resp = model.generate_content(prompt)
    class Obj:  # minimal adapter to keep downstream code unchanged
        class ChoicesMsg:
            def __init__(self, content):
                self.message = type("Msg", (), {"content": content})
        def __init__(self, text):
            self.choices = [self.ChoicesMsg(text)]
    return Obj(resp.text)

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [5]:
# Try it
question = "What is LangSmith used for? Answer for a product engineer."
ai_answer = langsmith_rag(question)
print(ai_answer)


LangSmith is a platform designed for product engineers to build and manage production-grade LLM applications. It allows you to monitor, trace, and evaluate your applications to ensure quality and reliability. Additionally, it offers tools to test prompts, providing automatic version control and collaboration features for faster iteration.
